In [1]:
import torch
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def generar_matriz_confusion(nivel_oclusion=0.5):
    y_true = []
    y_pred = []

    model.eval()
    print(f"⌛ Generando predicciones para oclusión al {nivel_oclusion*100}%...")

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)

            # Aplicar oclusión si es necesario
            if nivel_oclusion > 0:
                # Usamos el mismo parche central del script anterior
                inputs[:, :, 50:150, 50:150] = 0

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    # Crear la matriz
    cm = confusion_matrix(y_true, y_pred)

    # Visualización
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicción del Modelo')
    plt.ylabel('Clase Real')
    plt.title(f'Matriz de Confusión - Visibilidad Parcial ({nivel_oclusion*100}%)')

    # Guardar en Drive
    plt.savefig(f"{SAVE_DIR}/matriz_confusion_{int(nivel_oclusion*100)}.png")
    plt.show()

    # Imprimir reporte métrico (Precisión, Recall, F1-Score)
    print("\n📝 Reporte de Clasificación:")
    print(classification_report(y_true, y_pred, target_names=class_names))

# Ejecutar para el caso más crítico
generar_matriz_confusion(nivel_oclusion=0.5)